In [ ]:
!pip install corus
!pip install razdel
!pip install pymorphy2

In [ ]:
!pip install bigartm

In [ ]:
!pip install pyLDAvis==3.4.1

In [5]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [31]:
import re
import nltk
import pymorphy2
import numpy as np
from corus import load_lenta
from nltk.corpus import stopwords
from razdel import sentenize
from tqdm import tqdm_notebook
from pprint import pprint

import gensim
from gensim import corpora
from gensim.models import CoherenceModel

import matplotlib.pyplot as plt

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import artm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

random_state = 9

### Импорт данных

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

In [ ]:
data = [next(records).text for i in range(1000)]
data[5]

### Предобработка

#### Нормализация и удаление стоп слов

In [ ]:
morph = pymorphy2.MorphAnalyzer()
nltk.download('stopwords')

def normalization(data, stopwords = stopwords.words('russian')):
  text = re.sub('ё', 'е', data.lower())
  text = re.sub(r'([.,!?])', r' \1', text)
  text = re.sub(r'[^а-яА-Я\s]+', '', text)
  text = text.strip()
  text = [w for w in text.split() if w not in stopwords]
  text = [w for w in text if len(w) >= 2]

  return ' '.join(text)

#### Токенизация, Лемматизация

In [ ]:
text = data[5]
text = [_.text for _ in list(sentenize(text))]
text

In [13]:
def lem_tok(data, morph = morph):
  result = [morph.parse(x)[0].normal_form for x in data.split()]

  return result


#### Окончательная обработка


In [14]:
def get_result(data, morph = morph, stopwords = stopwords.words('russian')):
  result = normalization(data)
  result = lem_tok(result)
  return result

In [15]:
data_processed = [get_result(elem) for elem in data]

In [35]:
data_processed, x_test = train_test_split(data_processed, random_state=random_state)

### LDA

In [16]:
bigram = gensim.models.Phrases(data_processed, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_processed], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [17]:
def get_ngrams(texts_out):
    texts_out = [bigram_mod[doc] for doc in texts_out]
    texts_out = [trigram_mod[bigram_mod[doc]] for doc in texts_out]
    return texts_out

In [18]:
data_ready = get_ngrams(data_processed)

In [19]:
for item in data_ready:
  for token in item:
    if len(item) < 4 and '_' in token:
      print(item)

In [20]:
id2word = corpora.Dictionary(data_ready)

In [21]:
corpus = [id2word.doc2bow(text) for text in data_ready]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=4,
    random_state=random_state,
    update_every=1, 
    chunksize=30,
    passes=250, 
    alpha='symmetric', 
    iterations=500, 
    per_word_topics=True,
)

pprint(lda_model.print_topics())

In [23]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.264136 -0.061734       1        1  50.698979
0     -0.009074  0.216938       2        1  24.191877
1     -0.120342 -0.183233       3        1  16.492427
2     -0.134719  0.028029       4        1   8.616718, topic_info=           Term         Freq        Total Category  logprob  loglift
60       россия   726.000000   726.000000  Default  30.0000  30.0000
928    компания   248.000000   248.000000  Default  29.0000  29.0000
16          год  1236.000000  1236.000000  Default  28.0000  28.0000
796     процент   164.000000   164.000000  Default  27.0000  27.0000
1191     летний   202.000000   202.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1081  появиться    26.059508    68.619332   Topic4  -5.9522   1.4833
519         имя    25.573546    95.794763   Topic4  -5.9710   1.1308
107       самый    24.987481    87.648857   Topic4  -5.9942   1.1965
160     который    31.041675   926.324020   Topic4  -5.7773  -0.9444
1362      знать    23.841090    65.052841   Topic4  -6.0412   1.4477

[213 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
3811      3  0.985467  авиакомпания
1062      1  0.231616       аккаунт
1062      2  0.213800       аккаунт
1062      4  0.534499       аккаунт
2987      3  0.982333      аэропорт
...     ...       ...           ...
2986      1  0.183914     экономика
2986      3  0.802534     экономика
320       1  0.886578           это
320       2  0.113180           это
3228      4  0.960886          юзер

[235 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3])

In [ ]:
print('Perplexity: ', lda_model.log_perplexity(corpus))

#### Черновик визуализации метрик на темах

Тут проблема с List Index, которую я так и не смог решить. top = lda_model.get_topics()

print(len(top))

In [107]:
def get_coherence_mean(model, corpus, n_top_words = 9):
  coherence_model = CoherenceModel(
      model=model,
      corpus=corpus,
      coherence = 'u_mass'
  )
  coherence = coherence_model.get_coherence()
  return coherence

def compute_coherence_values(min_topic = 4,
                             max_topic = 12,
                             step = 3):
  coherence_values =[]
  model_list = []

  for num_topics in tqdm_notebook(range(min_topic, max_topic, step)):
    model = gensim.models.ldamodel.LdaModel(
        corpus=corpus,
        id2word=id2word,
        num_topics=num_topics,
        random_state=random_state,
        update_every=1, 
        chunksize=10, 
        passes=100, 
        alpha='symmetric', 
        iterations=100,
        per_word_topics=True
    )
    model_list.append(model)
    coherence_values.append(get_coherence_mean(model, x_test))

  return model_list, coherence_values




In [ ]:
model_list, coherence_values = compute_coherence_values(min_topic=min_topic,
                                                        max_topic=max_topic,
                                                        step = step)

In [ ]:
x = range(min_topic, max_topic, step)

plt.figure(figsize =(15, 7))

plt.plot(x, coherence_values)
plt.xlabel('Num_topics')
plt.ylabel('Coherence Score')

In [53]:
for n_topics, coherence in zip(x, coherence_values):
  print(f'Num topics = {n_topics}, Coherence = {coherence:.3f}')

Num topics = 1, Coherence = 0.538
Num topics = 2, Coherence = 0.511
Num topics = 3, Coherence = 0.540
Num topics = 4, Coherence = 0.543
Num topics = 5, Coherence = 0.562
Num topics = 6, Coherence = 0.559
Num topics = 7, Coherence = 0.556
Num topics = 8, Coherence = 0.549


### BigARTM

In [ ]:
data_processed = [' '.join(elem) for elem in data_processed]

In [ ]:
cv = CountVectorizer(max_features=1000, max_df=0.8, min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform(data_processed).todense()).T

In [ ]:
n_wd.shape

In [ ]:
token_list = [i for i in cv.vocabulary_.keys()]

In [ ]:
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=token_list)

In [ ]:
def fit_model_simple(seed=random_state):
  model = artm.ARTM(num_topics=40,
                    dictionary=bv.dictionary,
                    cache_theta=True,
                    seed=seed,
                    regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta',tau=-0.15),
                                  artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=1.5e+5)])

  model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))


  model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
  model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=100))

  model.fit_offline(bv, num_collection_passes=30)
  return model

In [ ]:
model = fit_model_simple()
model.score_tracker["perplexity_score"].last_value

In [ ]:
topics_to_check =  [10, 15, 20]

In [ ]:
def run_hyperparams_search():

  space={
       'num_topics': hp.choice('num_topics', topics_to_check),
        'phi_tau': hp.uniform('SparsePhi', -1, 1),
       'theta_tau': hp.uniform('SparseTheta', -1, 1),
       'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
  }

  trials = Trials()

  best_hyperparams = fmin(
      fn=objective,
      space=space,
      algo=tpe.suggest,
      max_evals=20,
      trials=trials,
  )
  return best_hyperparams

In [ ]:
def objective(space):
  model = artm.ARTM(num_topics=space['num_topics'], dictionary=bv.dictionary, cache_theta=True)

  model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))
  model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
  model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
  model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

  model.regularizers.add(
      artm.SmoothSparsePhiRegularizer(
        name='SparsePhi',
        tau=space['phi_tau']
    ),
  ) # Сглаживание распределений терминов в темах. Используется для выделения фоновых тем, собирающих общую лексику языка или общую лексику данной коллекции.
  model.regularizers.add(
      artm.SmoothSparseThetaRegularizer(
          name='SparseTheta',
          tau=space['theta_tau']
      ),
  ) # Сглаживание распределений тем в документах. Используется для выделения фоновых слов в каждом документах.
  model.regularizers.add(
      artm.DecorrelatorPhiRegularizer(
        name='DecorrelatorPhi',
        tau=space['decorrelation_tau']
      ),
  ) # Декоррелирование распределений терминов в темах. Используется для повышения различности лексических ядер предметных тем.

  model.fit_offline(bv, num_collection_passes=30)

  perplexity_score = model.score_tracker["perplexity_score"].last_value
  print("Perplexity:", perplexity_score)
  return {'loss': perplexity_score, 'status': STATUS_OK }

In [ ]:

best = run_hyperparams_search()

In [ ]:
best

In [ ]:
best = {'DecorrelatorPhi': 54323.627155284725,
 'SparsePhi': -0.41285940422957523,
 'SparseTheta': 0.03285979999746358,
 'num_topics': 2}

In [ ]:
def fit_model(seed=random_state):
  model = artm.ARTM(num_topics=topics_to_check[best['num_topics']], dictionary=bv.dictionary, cache_theta=True, seed=seed)
  model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                        dictionary=bv.dictionary))

  model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
  model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
  model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

  model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=best['SparsePhi'])) # сглаживание/разреживание матрицы Phi
  model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=best['SparseTheta'])) # сглаживание/разреживание матрицы Theta
  model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=best['DecorrelatorPhi'])) # сделать темы более разнообразными

  model.fit_offline(bv, num_collection_passes=30)
  return model

In [ ]:
model = fit_model()
model.score_tracker["perplexity_score"].last_value

338.640380859375

In [ ]:
def prepare_vis_data():
    phi = model.get_phi()
    theta = model.get_theta().to_numpy().T
    theta = theta / theta.sum(axis=1, keepdims=1)
    data = {'topic_term_dists': phi.to_numpy().T,
            'doc_topic_dists': theta,
            'doc_lengths': n_wd.sum(axis=0).tolist(),
            'vocab': phi.T.columns,
            'term_frequency': n_wd.sum(axis=1).tolist()}
    return data

In [ ]:
model_data = prepare_vis_data()
model_vis = pyLDAvis.prepare(**model_data)

In [ ]:
pyLDAvis.enable_notebook()
model_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.180111 -0.094866       1        1  11.715127
18     0.174743  0.036388       2        1   8.611588
11     0.130648  0.057824       3        1   8.527482
10     0.157324 -0.044946       4        1   7.602901
1      0.161787  0.156208       5        1   7.332848
8      0.160721 -0.119408       6        1   7.003060
17     0.100356  0.072338       7        1   6.843565
3      0.025120  0.177893       8        1   5.358032
9      0.036673 -0.207627       9        1   5.356799
14    -0.030061 -0.212099      10        1   5.099949
0     -0.016219  0.160897      11        1   4.561494
2     -0.067972 -0.144590      12        1   4.193212
7      0.005502  0.064797      13        1   3.986719
15    -0.136453 -0.121455      14        1   2.610108
13    -0.094527  0.014459      15        1   2.609804
16    -0.132412  0.054577      16        1   2.440447
12    -0.135890  0.083949      17        1   2.176395
4     -0.161011  0.071118      18        1   2.133685
5     -0.179221 -0.002728      19        1   1.031422
6     -0.179221 -0.002728      20        1   0.805363, topic_info=            Term   Freq       Total Category  logprob  loglift
114   соглашение  324.0  324.000000  Default     30.0     30.0
317       власть  268.0  268.000000  Default     29.0     29.0
431  сша дональд  240.0  240.000000  Default     28.0     28.0
295     инцидент  270.0  270.000000  Default     27.0     27.0
291         лига  243.0  243.000000  Default     26.0     26.0
..           ...    ...         ...      ...      ...      ...
16        житель    0.0  120.319275  Topic20     -inf     -inf
17     заявление    0.0  250.996937  Topic20     -inf     -inf
18        занять    0.0   45.432887  Topic20     -inf     -inf
19         пятый    0.0  227.800035  Topic20     -inf     -inf
20        работа    0.0   64.036536  Topic20     -inf     -inf

[775 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
534       3  1.006090        август
949       5  0.998612  авиакомпания
694       3  0.067113    автомобиль
694       7  0.933995    автомобиль
796       2  1.001142         автор
...     ...       ...           ...
302       1  0.101714          язык
302       2  0.323636          язык
302       4  0.573298          язык
364      15  1.004515         якобы
331       1  1.010780        япония

[930 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 19, 12, 11, 2, 9, 18, 4, 10, 15, 1, 3, 8, 16, 14, 17, 13, 5, 6, 7])